In [ ]:
import os
import pathlib
import sys
import time

import numpy as np
import pandas as pd
import psutil

cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd
else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break
sys.path.append(str(root_dir / "utils"))
from arg_parsing_utils import check_for_missing_args, parse_args
from notebook_init_utils import bandicoot_check, init_notebook

root_dir, in_notebook = init_notebook()

sys.path.append(f"{root_dir}/3.cellprofiling/featurization_utils/")
from area_size_shape_utils import measure_3D_area_size_shape
from area_size_shape_utils_gpu import measure_3D_area_size_shape_gpu
from featurization_parsable_arguments import parse_featurization_args
from loading_classes import ImageSetLoader, ObjectLoader
from resource_profiling_util import get_mem_and_time_profiling

In [ ]:
if not in_notebook:
    arguments_dict = parse_featurization_args()
    patient = arguments_dict["patient"]
    well_fov = arguments_dict["well_fov"]
    compartment = arguments_dict["compartment"]
    channel = arguments_dict["channel"]
    processor_type = arguments_dict["processor_type"]

else:
    well_fov = "F3-1"
    patient = "NF0014_T1"
    compartment = "Organoid"
    channel = "DNA"
    processor_type = "CPU"

image_set_path = pathlib.Path(
    f"{root_dir}/data/{patient}/profiling_input_images/{well_fov}/"
)

output_parent_path = pathlib.Path(
    f"{root_dir}/data/{patient}/extracted_features/{well_fov}/"
)
output_parent_path.mkdir(parents=True, exist_ok=True)

In [3]:
channel_n_compartment_mapping = {
    "DNA": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "Nuclei": "nuclei_",
    "Cell": "cell_",
    "Cytoplasm": "cytoplasm_",
    "Organoid": "organoid_",
}

In [4]:
start_time = time.time()
# get starting memory (cpu)
start_mem = psutil.Process(os.getpid()).memory_info().rss / 1024**2

In [5]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    anisotropy_spacing=(1, 0.1, 0.1),
    channel_mapping=channel_n_compartment_mapping,
)

In [ ]:
object_loader = ObjectLoader(
    image_set_loader.image_set_dict["DNA"],
    image_set_loader.image_set_dict[compartment],
    "DNA",
    compartment,
)


# area, size, shape
if processor_type == "GPU":
    size_shape_dict = measure_3D_area_size_shape_gpu(
        image_set_loader=image_set_loader,
        object_loader=object_loader,
    )
elif processor_type == "CPU":
    size_shape_dict = measure_3D_area_size_shape(
        image_set_loader=image_set_loader,
        object_loader=object_loader,
    )
else:
    raise ValueError(
        f"Processor type {processor_type} is not supported. Use 'CPU' or 'GPU'."
    )

In [ ]:
final_df = pd.DataFrame(size_shape_dict)

# prepend compartment and channel to column names
for col in final_df.columns:
    if col not in ["object_id"]:
        final_df[col] = final_df[col].astype(np.float32)
        final_df.rename(
            columns={col: f"Area.Size.Shape_{compartment}_{col}"},
            inplace=True,
        )

final_df.insert(1, "image_set", image_set_loader.image_set_name)

output_file = pathlib.Path(
    output_parent_path
    / f"AreaSizeShape_{compartment}_{processor_type}_features.parquet"
)
final_df.to_parquet(output_file, index=False)
final_df.head()

,object_id,image_set,Area.Size.Shape_Cytoplasm_VOLUME,Area.Size.Shape_Cytoplasm_CENTER.X,Area.Size.Shape_Cytoplasm_CENTER.Y,Area.Size.Shape_Cytoplasm_CENTER.Z,Area.Size.Shape_Cytoplasm_BBOX.VOLUME,Area.Size.Shape_Cytoplasm_MIN.X,Area.Size.Shape_Cytoplasm_MAX.X,Area.Size.Shape_Cytoplasm_MIN.Y,Area.Size.Shape_Cytoplasm_MAX.Y,Area.Size.Shape_Cytoplasm_MIN.Z,Area.Size.Shape_Cytoplasm_MAX.Z,Area.Size.Shape_Cytoplasm_EXTENT,Area.Size.Shape_Cytoplasm_EULER.NUMBER,Area.Size.Shape_Cytoplasm_EQUIVALENT.DIAMETER,Area.Size.Shape_Cytoplasm_SURFACE.AREA
0,31,E11-2,22888.0,663.253784,484.497070,4.156239,213465.0,614.0,721.0,435.0,540.0,0.0,19.0,0.107221,7.0,35.226521,927.302246
1,63,E11-2,119019.0,1089.716675,528.066956,14.317916,545832.0,1028.0,1180.0,452.0,585.0,0.0,27.0,0.218051,-21.0,61.029419,2103.285889
2,95,E11-2,44721.0,1111.264282,578.127808,4.351289,212976.0,1075.0,1162.0,509.0,653.0,0.0,17.0,0.209981,-11.0,44.039017,987.116211
3,127,E11-2,146721.0,1034.049927,617.990662,5.857887,751842.0,959.0,1112.0,533.0,715.0,0.0,27.0,0.195149,-40.0,65.438179,2652.166748
4,159,E11-2,61614.0,1312.189453,1044.139648,11.978316,387072.0,1238.0,1366.0,984.0,1096.0,0.0,27.0,0.159180,-20.0,49.003479,1509.555664


In [ ]:
end_mem = psutil.Process(os.getpid()).memory_info().rss / 1024**2
end_time = time.time()
get_mem_and_time_profiling(
    start_mem=start_mem,
    end_mem=end_mem,
    start_time=start_time,
    end_time=end_time,
    feature_type="AreaSizeShape",
    well_fov=well_fov,
    patient_id=patient,
    channel="DNA",
    compartment=compartment,
    CPU_GPU=processor_type,
    output_file_dir=pathlib.Path(
        f"{root_dir}/data/{patient}/extracted_features/run_stats/{well_fov}_AreaSizeShape_DNA_{compartment}_{processor_type}.parquet"
    ),
)


        Memory and time profiling for the run:

        Patient ID: NF0014

        Well and FOV: E11-2

        Feature type: AreaSizeShape

        CPU/GPU: CPU")

        Memory usage: 867.08 MB

        Time:

        --- %s seconds --- % 152.22045707702637

        --- %s minutes --- % 2.5370076179504393

        --- %s hours --- % 0.04228346029917399
    


True